In [ ]:
!pip install onnxruntime
!pip install git+https://github.com/onnx/tensorflow-onnx
!pip install -U tf2onnx

In [ ]:
import onnxruntime
from scipy.special import softmax
from PIL import Image
from torchvision.transforms import ToTensor
import os
import numpy as np
import tensorflow as tf

In [ ]:
!python -m tf2onnx.convert --saved-model /content/drive/MyDrive/CV_project/안구질환_모델/개_안구_all_ttv_au_under_250000_5class.tf --output /content/drive/MyDrive/CV_project/안구질환_모델/ResNet_squared_all_5class_code/개_안구_all_ttv_au_under_250000_5class.onnx --opset 12

2024-06-06 03:50:07.374679: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-06 03:50:07.374727: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-06 03:50:07.376596: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-06 03:50:08.382638: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2

In [ ]:
# prompt: /content/Resnet_squared_all_4class_b64_e40.onnx 이 파일을  /content/drive/MyDrive/CV_project/안구질환_모델/ResNet_squared_all_4class_code 이 경로의 저장하는 방법을 알려줘

# Import the necessary libraries
import shutil

# Define the source and destination paths
source_path = "/content/Resnet_squared_all_4class_b64_e40.onnx"
destination_path = "/content/drive/MyDrive/CV_project/안구질환_모델/ResNet_squared_all_4class_code/Resnet_squared_all_4class_b64_e40.onnx"

# Copy the file
shutil.copyfile(source_path, destination_path)

# Print a message to confirm the file has been copied
print("File copied successfully.")


File copied successfully.


## onnx 추론


In [ ]:
onnx_path = '/content/drive/MyDrive/CV_project/안구질환_모델/ResNet_squared_all_5class_25000_code/개_안구_all_ttv_au_under_250000_5class.onnx'

In [ ]:
ort_session = onnxruntime.InferenceSession(onnx_path)
input_name = ort_session.get_inputs()[0].name

In [ ]:
tf_path = '/content/drive/MyDrive/CV_project/안구질환_모델/개_안구_all_ttv_au_under_250000_5class.tf'
tf_model = tf.keras.models.load_model(tf_path)

In [ ]:
sick_img_path = "/content/drive/MyDrive/CV_project/data/안구질환/개_train_test_val_squared/test/궤양성각막질환/crop_D0_125a8627-60a5-11ec-8402-0a7404972c70.jpg"

normal_img_path = "/content/drive/MyDrive/CV_project/data/안구질환/개_train_test_val_squared/test/정상/crop_D0_03b2ab5b-60a5-11ec-8402-0a7404972c70.jpg"

img_path = sick_img_path


In [ ]:
img = Image.open(img_path)
img = img.resize((224, 224))
img = np.array(img, dtype=np.float32)

img = img / 255.
img = img.reshape((1, 224, 224, 3))
ort_inputs = {input_name: img}
ort_output = ort_session.run(None, ort_inputs)
ort_output

[array([[1.0000000e+00, 4.4595357e-15, 9.6807660e-16, 5.0382475e-14,
         1.0349237e-09]], dtype=float32)]

In [ ]:
img = Image.open(img_path)
img = img.resize((224, 224))
img = np.array(img)

img = img / 255.
img = tf.expand_dims(img, axis=0)
tf_model.predict(img, verbose=0)

array([[1.0000000e+00, 4.4554208e-15, 9.6831292e-16, 5.0310738e-14,
        1.0329477e-09]], dtype=float32)

In [ ]:
labels= [
    '궤양성각막질환',
    '백내장',
    '안검종양',
    '핵경화'
]
labels[np.argmax(ort_output[0][0])]

'궤양성각막질환'